In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')

In [2]:
pizza_df = pd.read_excel('dataset/master/Pizza_Sale.xlsx')
ingred_df = pd.read_excel('dataset/master/Pizza_ingredients.xlsx')

In [3]:
def parse_dates(date):
  for fmt in ('%d-%m-%Y', '%d/%m/%Y'):
    try:
      return pd.to_datetime(date, format=fmt)
    except ValueError:
      pass
  raise ValueError(f'no valid date format found for {date}')

In [4]:
pizza_df['order_date'] = pizza_df['order_date'].apply(parse_dates)

In [5]:
pizza_df = pizza_df[['order_date', 'pizza_name', 'quantity']]

In [6]:
sales_summary = pizza_df.groupby(['order_date', 'pizza_name']).sum().reset_index()

In [7]:
sales_summary.head()

,order_date,pizza_name,quantity
0,2015-01-01,The Barbecue Chicken Pizza,11
1,2015-01-01,The Big Meat Pizza,5
2,2015-01-01,The Calabrese Pizza,1
3,2015-01-01,The California Chicken Pizza,4
4,2015-01-01,The Chicken Alfredo Pizza,2


In [8]:
sales_pivot = sales_summary.pivot(index='order_date', columns='pizza_name', values='quantity').fillna(0)

In [9]:
arima_models = {}

for pizza_name in sales_pivot.columns:
  try:
    model = ARIMA(sales_pivot[pizza_name], order=(1, 1, 0))
    model_fit = model.fit()
    arima_models[pizza_name] = model_fit
  except:
    print(f'ARIMA model for {pizza_name} failed to fit')

In [10]:
prediction_days = 7
predictions_arima = {}

for pizza_name, model in arima_models.items():
  predictions_arima[pizza_name] = model.predict(start=len(sales_pivot), end=len(sales_pivot) + prediction_days - 1)

In [11]:
predictions_df = pd.DataFrame(predictions_arima)

In [12]:
predictions_df.index = pd.date_range(start=sales_pivot.index[-1], periods=prediction_days, freq='D')

In [13]:
ingredients_df = ingred_df[['pizza_name', 'pizza_ingredients', 'Items_Qty_In_Grams']]

In [14]:
ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)

In [15]:
predictions_df

,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
2015-12-31,8.163282,4.359358,0.888569,1.432509,7.0,2.418361,2.102540,7.119636,2.906248,7.042489,...,3.440105,2.285151,2.036685,5.912097,6.099148,3.448682,2.0,3.178298,3.0,7.835581
2016-01-01,9.006665,5.256593,1.506208,2.251518,7.0,3.252222,3.002629,8.502385,4.002197,8.000451,...,4.251196,3.759232,2.500673,5.496137,7.301283,4.250878,2.0,4.007948,3.0,8.772523
2016-01-02,8.619401,4.765913,1.162977,1.823589,7.0,2.812600,2.575658,7.838582,3.428536,7.531646,...,3.829457,2.958851,2.277190,5.685834,6.803109,3.836058,2.0,3.630104,3.0,8.366936
2016-01-03,8.797225,5.034256,1.353716,2.047181,7.0,3.044374,2.778198,8.157248,3.728812,7.761069,...,4.048746,3.393434,2.384832,5.599323,7.009556,4.050564,2.0,3.802184,3.0,8.542508
2016-01-04,8.715571,4.887504,1.247719,1.930355,7.0,2.922180,2.682120,8.004269,3.571636,7.648794,...,3.934724,3.157468,2.332985,5.638776,6.924003,3.939641,2.0,3.723814,3.0,8.466505
2016-01-05,8.753065,4.967760,1.306623,1.991396,7.0,2.986602,2.727696,8.077708,3.653908,7.703739,...,3.994012,3.285590,2.357958,5.620784,6.959457,3.997000,2.0,3.759506,3.0,8.499406
2016-01-06,8.735849,4.923870,1.273889,1.959502,7.0,2.952638,2.706076,8.042453,3.610844,7.676850,...,3.963184,3.216024,2.345930,5.628989,6.944764,3.967340,2.0,3.743251,3.0,8.485164


In [16]:
ingredients_df.head()

,pizza_name,pizza_ingredients,items_qty
0,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,The Barbecue Chicken Pizza,Red Peppers,15.0
2,The Barbecue Chicken Pizza,Green Peppers,20.0
3,The Barbecue Chicken Pizza,Tomatoes,30.0
4,The Barbecue Chicken Pizza,Red Onions,60.0


In [17]:
ingredient_quantities = {}

# Iterate through each pizza in the predictions
for pizza_name in predictions_df.columns:
  # Get the predicted quantity for the pizza
  predicted_quantity = predictions_df[pizza_name].sum()

  # Get the ingredients for the pizza
  pizza_ingredients = ingredients_df[ingredients_df['pizza_name'] == pizza_name]

  # Iterate through each ingredient for the pizza
  for index, row in pizza_ingredients.iterrows():
    ingredient = row['pizza_ingredients']
    ingredient_qty = row['items_qty']

    # Calculate the required quantity of the ingredient
    required_quantity = predicted_quantity * ingredient_qty

    # Add the required quantity to the dictionary
    if ingredient not in ingredient_quantities:
      ingredient_quantities[ingredient] = 0
    ingredient_quantities[ingredient] += required_quantity




In [18]:
ingredient_requirements_df = pd.DataFrame.from_dict(ingredient_quantities, orient='index', columns=['quantity'])

In [19]:
ingredient_requirements_df

,quantity
Barbecued Chicken,5471.195153
Red Peppers,11349.929940
Green Peppers,8096.027201
Tomatoes,34840.939745
Red Onions,54748.422229
...,...
Luganega Sausage,2646.689882
Onions,1323.344941
Soppressata Salami,4870.875692
Peperoncini verdi,1441.239606


In [21]:
print(ingredient_requirements_df.to_string())

                                quantity
Barbecued Chicken            5471.195153
Red Peppers                 11349.929940
Green Peppers                8096.027201
Tomatoes                    34840.939745
Red Onions                  54748.422229
Barbecue Sauce               1823.731718
Bacon                       20179.703935
Pepperoni                   23912.964809
Italian Sausage               341.952555
Chorizo Sausage              1709.762775
Brie Carre Cheese             262.191032
Prosciutto                    262.191032
Caramelized Onions                   NaN
Pears                          87.397011
Thyme                          43.698505
Garlic                      17843.077799
?duja Salami                 1612.325924
Pancetta                     2418.488886
Friggitello Peppers           403.081481
Chicken                     44563.749754
Artichokes                   8247.815640
Spinach                     21052.538164
Jalapeno Peppers             3738.624697
Fontina Cheese  